# Hyperparameter Search

Before evaluating a final model, we will first go through a hyperparamter search for the models. To reduce computational waste, we will only be training and evaluating hyperparamters using the LOPO strategy on Kka2, a protein with a relatively small number of samples but a good balance of labels. The hyperparamters will we be selecting for are:

## Hyperparameters
- Number of hidden layers
- Number of nodes in each hidden layer
- Labeling Thresholds

Attempts at normalization and regularization didn't work with our current data in initial baseline runs, so they have been exlcuded from this analysis.

## Analysis Criterion
To choose our final hyperparameters, we will identify the model with the best precision and accuracy score.

In [1]:
# Importing 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import optimizers, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [2]:
# Reading in Data
data_path = 'data/merged.csv'
data = pd.read_csv(data_path)

In [8]:
# Data preparation functions

def label_type(row, thresh_offset):
    """ Converts continuous label to categorical label
    """
    if row['scaled_effect'] < 1 - thresh_offset:
        return('Deleterious')
    elif row['scaled_effect'] > 1 + thresh_offset:
        return('Beneficial')
    else:
        return('Netural')
    
def lopo_train_test_split(protein, curr_data):
    train_data = curr_data[data.protein != protein].drop(['protein', 'pdb', 'resnum'], axis=1)
    test_data = curr_data[data.protein == protein].drop(['protein', 'pdb', 'resnum'], axis=1)
    
    
    y_train = train_data.type
    encoder_train = LabelEncoder()
    encoder_train.fit(y_train)
    y_train = to_categorical(encoder_train.transform(y_train))
    
    x_train = train_data.drop(['type'], axis=1)
    
    
    
    
    y_test = test_data.type
    encoder_test = LabelEncoder()
    encoder_test.fit(y_test)
    y_test = to_categorical(encoder_test.transform(y_test))
    
    x_test = test_data.drop(['type'], axis=1)

    return x_train, y_train, x_test, y_test

In [40]:
# Model definitions
def nn_model(num_layers, num_nodes):
    model = Sequential()
    inputs = Input(shape=(968,))
    x = Dense(num_nodes, activation=tf.nn.relu)(inputs)
    for layers in range(num_layers-1):
        x = Dense(num_nodes, activation=tf.nn.relu)(inputs)
    outputs = Dense(3, activation=tf.nn.softmax)(x)
    opt = optimizers.Adam(learning_rate = 0.1)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=[tf.keras.metrics.CategoricalAccuracy(),
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall()])
    return(model)

In [ ]:
# Hyperparamter List
num_hl = [2, 4, 8, 16, 32]
num_nodes = [25, 100, 400, 800]
label_thresholds = [.025, .05, .1, .2]

# Other Variables to define
protein = 'Uba1'
column_list = ['Hidden Layers', 'Number Nodes', 'Threshold Offset', 'Loss', 'Accuracy', 'Precision', 'Recall']

# Evaluation Metric Storage
eval_metrics = pd.DataFrame(columns=column_list)

for hl in num_hl:
    for nodes in num_nodes:
        for thresholds in label_thresholds:
            # Generate the Split Training Set
            data['type'] = data.apply(lambda row: label_type(row, thresholds), axis = 1)
            data_final = data.drop(['scaled_effect'], axis=1)
            x_train, y_train, x_test, y_test = lopo_train_test_split(protein, data_final)
            
            # Build the Model
            print("Current Model: HL - {} | Nodes - {} | Threshold Offset - {}".format(hl, nodes, thresholds))
            curr_model = nn_model(hl, nodes)
            curr_model.fit(x_train, y_train, epochs = 50, batch_size = 10, verbose=1)
            
            # Calculate Evaluation Metrics
            loss, acc, prec, rec = curr_model.evaluate(x_test, y_test)
#             probs = curr_model.predict_proba(x_test)
#             predictions = curr_model.predict(x_test)
#             precision = precision_score(y_test, predictions, average="macro", zero_division = 0)
#             score = curr_model.score(x_test, y_test)
            
            # Append to Eval Storage
            eval_metrics = eval_metrics.append(pd.DataFrame([[hl, nodes, thresholds, loss, acc, prec, rec]], columns=column_list))
            
eval_metrics.to_csv('hyperparamter-eval.csv')

Current Model: HL - 2 | Nodes - 25 | Threshold Offset - 0.025
Train on 48217 samples
Epoch 1/50
48217/48217 [==============================] - 31s 641us/sample - loss: 3.8274 - categorical_accuracy: 0.5731 - precision_14: 0.5730 - recall_14: 0.5332
Epoch 2/50
48217/48217 [==============================] - 30s 616us/sample - loss: 0.9844 - categorical_accuracy: 0.5738 - precision_14: 0.5728 - recall_14: 0.5203
Epoch 3/50
48217/48217 [==============================] - 30s 622us/sample - loss: 0.9849 - categorical_accuracy: 0.5738 - precision_14: 0.5743 - recall_14: 0.5200
Epoch 4/50
48217/48217 [==============================] - 30s 630us/sample - loss: 0.9843 - categorical_accuracy: 0.5738 - precision_14: 0.5723 - recall_14: 0.5203
Epoch 5/50
48217/48217 [==============================] - 30s 623us/sample - loss: 0.9841 - categorical_accuracy: 0.5733 - precision_14: 0.5731 - recall_14: 0.5317
Epoch 6/50
48217/48217 [==============================] - 30s 627us/sample - loss: 0.9839 - cat